<a href="https://colab.research.google.com/github/seeedata/BADA/blob/main/%EB%88%84%EC%88%98%20%ED%83%90%EC%A7%80%20%EA%B8%B0%EC%97%85%EC%9D%98%20%EC%8B%A0%EC%82%AC%EC%97%85%20%ED%9A%A8%EC%9A%A9%EC%84%B1%20%ED%8F%89%EA%B0%80/SVM%20%EB%B6%84%EB%A5%98%20%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bayesian-optimization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from bayes_opt import BayesianOptimization
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_m = pd.read_csv('/content/drive/MyDrive/센서 데이터.csv', encoding='cp949')
df_m.head(10)

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,Y
0,가격,11400.0000,11100.0000,14400.0000,12300.0000,15300.0000,13200.0000,11100.0000,4500.0000,6300.0000,...,8700.0000,11700.0000,13500.0000,4500.0000,11100.0000,11400.0000,9600.0000,9900.0000,9000.0000,NaN
1,1,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,N
2,2,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,N
3,3,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,N
4,4,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,N
5,5,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,N
6,6,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,N
7,7,0.0317,0.0956,0.1321,0.1408,0.1674,0.1710,0.0731,0.1401,0.2083,...,0.0201,0.0248,0.0131,0.0070,0.0138,0.0092,0.0143,0.0036,0.0103,N
8,8,0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465,...,0.0081,0.0120,0.0045,0.0121,0.0097,0.0085,0.0047,0.0048,0.0053,N
9,9,0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684,...,0.0145,0.0128,0.0145,0.0058,0.0049,0.0065,0.0093,0.0059,0.0022,N


In [ ]:
# 첫 번째 행 추출
first_row = df_m.iloc[0]

# 새로운 데이터 프레임 생성
price_df = pd.DataFrame([first_row])

In [ ]:
price_df

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,Y
0,가격,11400.0,11100.0,14400.0,12300.0,15300.0,13200.0,11100.0,4500.0,6300.0,...,8700.0,11700.0,13500.0,4500.0,11100.0,11400.0,9600.0,9900.0,9000.0,NaN


In [ ]:
df = df_m.drop(0, axis=0)
df.iloc[:, -1] = df.iloc[:, -1].map({'Y': 1, 'N': 0})

In [ ]:
df

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,Y
1,1,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
2,2,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
3,3,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
4,4,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
5,5,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,204,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,1
205,205,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,1
206,206,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,1
207,207,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,1


In [ ]:
X = df.iloc[:, 1:-1]
X

,1,2,3,4,5,6,7,8,9,10,...,51,52,53,54,55,56,57,58,59,60
1,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
2,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
3,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
4,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
5,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
205,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
206,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
207,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


In [ ]:
y = df.iloc[:, -1]
y

1      0
2      0
3      0
4      0
5      0
      ..
204    1
205    1
206    1
207    1
208    1
Name: Y, Length: 208, dtype: int64

# SVM

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

# 데이터 로드와 준비
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 결과를 저장할 최종 데이터 프레임
final_results = pd.DataFrame()

kernels = ['linear']
C_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

# FPR과 FNR을 계산하는 함수
def calculate_fpr_fnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    return fpr, fnr

# 1개의 특성부터 60개의 특성까지 반복
for n_features in range(1, 61):
    results = []

    for kernel in kernels:
        for C in C_values:
            svc = SVC(kernel=kernel, C=C)
            # RFE: Recursive Feature Elimination. 모델을 활용하여 가장 중요한 센서들을 선택
            rfe = RFE(estimator=svc, n_features_to_select=n_features)
            rfe.fit(X_train, y_train)
            # 선택된 주요 센서들로 X_train과 X_test를 변환하여 새로운 데이터셋 생성
            X_train_rfe = rfe.transform(X_train)
            X_test_rfe = rfe.transform(X_test)
            # SVC를 X_train_rfe와 y_train에 맞춰 학습시킴
            svc.fit(X_train_rfe, y_train)
            y_pred = svc.predict(X_test_rfe)

            accuracy = accuracy_score(y_test, y_pred)
            fpr, fnr = calculate_fpr_fnr(y_test, y_pred)

            # rfe.support_: RFE에 의해 해당 센서가 선택되었는지 여부
            selected_features = X_train.columns[rfe.support_].tolist()
            result = {
                'Number of Features': n_features,
                'Kernel': kernel,
                'C': C,
                'Selected Features': ', '.join(selected_features),
                'Accuracy': accuracy,
                'FPR': fpr,
                'FNR': fnr
            }
            results.append(result)

    # 상위 5개의 유니크한 조합을 선택
    results_df = pd.DataFrame(results)
    results_df.sort_values('Accuracy', ascending=False, inplace=True)
    top_results = pd.DataFrame()

    # 유니크한 조합들을 추출
    seen_combinations = set()
    for _, row in results_df.iterrows():
        features_key = row['Selected Features']
        # 이전에 보지 않았던 센서 조합인 경우에만 아래의 작업을 수행
        if features_key not in seen_combinations:
            top_results = top_results.append(row, ignore_index=True)
            seen_combinations.add(features_key)
        if len(top_results) >= 5:
            break

    final_results = pd.concat([final_results, top_results], axis=0)

# 특성의 수가 적은 순으로 최종 데이터 프레임 정렬
final_results.sort_values('Number of Features', ascending=True, inplace=True)
final_results.reset_index(drop=True, inplace=True)

# 최종 결과 출력
print(final_results)

<ipython-input-47-f0becd4cc8f3>:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_results = top_results.append(row, ignore_index=True)
<ipython-input-47-f0becd4cc8f3>:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_results = top_results.append(row, ignore_index=True)
<ipython-input-47-f0becd4cc8f3>:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_results = top_results.append(row, ignore_index=True)
<ipython-input-47-f0becd4cc8f3>:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_results = top_results.append(row, ignore_index=True)
<ipython-input-47-f0becd4cc8f3>:66: FutureWarning: The frame.append method is deprecated

     Number of Features  Kernel           C  \
0                     1  linear     10.0000   
1                     1  linear    100.0000   
2                     1  linear  10000.0000   
3                     1  linear   1000.0000   
4                     1  linear      0.0001   
..                  ...     ...         ...   
291                  59  linear  10000.0000   
292                  59  linear   1000.0000   
293                  59  linear      1.0000   
294                  59  linear     10.0000   
295                  60  linear      1.0000   

                                     Selected Features  Accuracy       FPR  \
0                                                   11  0.714286  0.428571   
1                                                   49  0.666667  0.380952   
2                                                   51  0.666667  0.380952   
3                                                   52  0.571429  0.333333   
4                                            

<ipython-input-47-f0becd4cc8f3>:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_results = top_results.append(row, ignore_index=True)


In [ ]:
final_results

,Number of Features,Kernel,C,Selected Features,Accuracy,FPR,FNR
0,1,linear,10.0000,11,0.714286,0.428571,0.142857
1,1,linear,100.0000,49,0.666667,0.380952,0.285714
2,1,linear,10000.0000,51,0.666667,0.380952,0.285714
3,1,linear,1000.0000,52,0.571429,0.333333,0.523810
4,1,linear,0.0001,36,0.500000,1.000000,0.000000
...,...,...,...,...,...,...,...
291,59,linear,10000.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.809524,0.142857,0.238095
292,59,linear,1000.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.761905,0.238095,0.238095
293,59,linear,1.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.857143,0.285714,0.000000
294,59,linear,10.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.809524,0.238095,0.142857


In [ ]:
# prices_series의 인덱스를 0부터 시작하도록 조정
prices_series = price_df.iloc[0, 1:-1]
prices_series.index = range(0, len(prices_series))

# 'Selected Features' 열에서 각 특성 번호에 해당하는 가격을 찾아 합산하는 함수
def get_total_price(selected_features_str):
    # 선택된 특성 번호를 정수 리스트로 변환
    # pandas의 인덱싱이 0부터 시작하기 때문에 1을 빼서 맞춰줌.
    selected_features = [int(feat) - 1 for feat in selected_features_str.split(', ')]
    # 해당 특성 번호에 대응하는 가격을 prices_series에서 찾아 합산
    total_price = prices_series.iloc[selected_features].sum()
    return total_price

# 'Total Price' 열을 계산하여 final_results 데이터프레임에 추가
final_results['Total Price'] = final_results['Selected Features'].apply(get_total_price)

final_results.head()

,Number of Features,Kernel,C,Selected Features,Accuracy,FPR,FNR,Total Price
0,1,linear,10.0000,11,0.714286,0.428571,0.142857,23400.0
1,1,linear,100.0000,49,0.666667,0.380952,0.285714,11100.0
2,1,linear,10000.0000,51,0.666667,0.380952,0.285714,14400.0
3,1,linear,1000.0000,52,0.571429,0.333333,0.523810,8700.0
4,1,linear,0.0001,36,0.500000,1.000000,0.000000,8100.0


In [ ]:
def calculate_total_cost(row):
    fnr = row['FNR']
    fpr = row['FPR']
    total_price = row['Total Price']

    # fpr가 1에 매우 가까운 경우를 처리
    if fpr >= 0.999:  # fpr가 0.999 이상일 경우, fpr를 조금 줄여서 0으로 나누는 상황 방지
        fpr = 0.999

    total_cost = 5000000 * 100 * fnr + 110000 * (100 - 100 * fnr) * fpr / (1 - fpr) + total_price * 5000 / 12
    return total_cost

# pro_rbf_df에 Total Cost 열 추가
final_results['Total Cost']  = final_results.apply(calculate_total_cost, axis=1)

In [ ]:
final_results

,Number of Features,Kernel,C,Selected Features,Accuracy,FPR,FNR,Total Price,Total Cost
0,1,linear,10.0000,11,0.714286,0.428571,0.142857,23400.0,8.825000e+07
1,1,linear,100.0000,49,0.666667,0.380952,0.285714,11100.0,1.523173e+08
2,1,linear,10000.0000,51,0.666667,0.380952,0.285714,14400.0,1.536923e+08
3,1,linear,1000.0000,52,0.571429,0.333333,0.523810,8700.0,2.681488e+08
4,1,linear,0.0001,36,0.500000,1.000000,0.000000,8100.0,1.099237e+10
...,...,...,...,...,...,...,...,...,...
291,59,linear,10000.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.809524,0.142857,0.238095,694500.0,4.098194e+08
292,59,linear,1000.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.761905,0.238095,0.238095,694200.0,4.109167e+08
293,59,linear,1.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.857143,0.285714,0.000000,693900.0,2.935250e+08
294,59,linear,10.0000,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",0.809524,0.238095,0.142857,688200.0,3.611250e+08
